# Traitement du corpus TCOF - CNRTL

**Imports**

In [39]:
import os
import pandas as pd
import re
import requests
import xml.etree.ElementTree as ET
import numpy as np
from bs4 import BeautifulSoup
import chardet

# **Téléchargement des corpus depuis le site CNRTL**

1. Depuis le site du CNRTL, télécharge chaque corpus dans un fichier compressé .zip (avec les fichiers .trs, .xml, .wav), dans le dossier indiqué par la variable filename
2. nomme chaque fichier compressé le nom du corpus, récupéré depuis la page html du site

In [ ]:
def corpus_name_cnrtl(number):
    url = "https://tcof.atilf.fr/index.php?r=corpus%2Fview&id={}".format(number)
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    # Find the h1 tag and extract its text
    h1_tag = soup.find('h1')
    text = h1_tag.get_text()
    # Extract the corpus name from the text
    corpus_name = text.replace('Corpus ', '')
    return corpus_name

In [ ]:
for i in range(1,800):
    try:
        url = 'https://tcof.atilf.fr/index.php?r=corpus/download-corpus&id={}'.format(i)
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            filename = 'E:/Corpus/TCOF_CNRTL/{}.zip'.format(corpus_name_cnrtl(i))
            with open(filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except:
        pass

# **Fichiers dans le dossier**
> **Avant de lancer**: dézipper manuellement tous les fichier zip 

Le script : 
1. Recense les fichiers existants pour chaque corpus
2. Output un dataframe avec pour chaque corpus, les fichiers existants

In [2]:
def get_files_cnrtl(path):
    list_corpus_path = [add_path2corpus(path, corpus) for corpus in get_all_folder(path)]
    mydict = {}
    for corpus_path in list_corpus_path:
        dict_corpus = get_corpus_files(corpus_path)
        mydict[corpus_path]=dict_corpus
    df = pd.DataFrame.from_dict(mydict, orient='index').rename_axis('corpus').reset_index()
    return df

In [3]:
def get_corpus_files(path):
    files = {}
    for file in os.listdir(path):
        if file.endswith('.xml'):
            files['xml']=file
        if file.endswith('.trs'):
            pattern = r'(anonymise_[\S]*.trs)'
            match = re.search(pattern, file)
            if match:
                files['anonymise_trs']=file
            else:
                files['trs']=file
        if file.endswith('.wav'):
            pattern = r'(anonymise_[\S]*.wav)'
            match = re.search(pattern, file)
            if match:
                files['anonymise_wav']=file
            else:
                files['wav']=file
        else:
            continue
    return files

In [4]:
def add_path2corpus(path,corpus):
    new = '{}\{}'.format(path, corpus)
    return new

In [5]:
def get_all_folder(path):
    directory = os.listdir(path)
    folder_list = []
    for folder in directory:
        if os.path.isdir(os.path.join(path, folder)):
            folder_list.append(folder)
    return folder_list

In [6]:
def add_corpus_names_col(df):
    wav = df['xml'].tolist()
    corpus_names = []
    for file in wav:
        try: 
            pattern = r'^(.*)\.xml$'
            match = re.search(pattern, file)
            if match:
                corpus = match.group(1)
        except:
            corpus = 'NAN'
        corpus_names.append(corpus)
    df['corpus_name'] = corpus_names
    return df

In [7]:
#mettre le chemin du dossier 
path_cnrtl = r'E:\Corpus\TCOF_CNRTL'
#obtenir le dataframe avec le noms des fichiers du dossier dedans
df_cnrtl = get_files_cnrtl(path_cnrtl)

In [9]:
df_cnrtl = add_corpus_names_col(df_cnrtl)

In [10]:
#liste de tous les fichiers pour chaque corpus
df_cnrtl 

,corpus,xml,wav,trs,anonymise_wav,anonymise_trs,corpus_name
0,E:\Corpus\TCOF_CNRTL\theorielinguistique_07,theorielinguistique_07.xml,theorielinguistique_07.wav,theorielinguistique_07.trs,NaN,NaN,theorielinguistique_07
1,E:\Corpus\TCOF_CNRTL\testssncf_qab_12,testssncf_qab_12.xml,testssncf_qab_12.wav,testssncf_qab_12.trs,anonymise_testssncf_qab_12.wav,anonymise_testssncf_qab_12.trs,testssncf_qab_12
2,E:\Corpus\TCOF_CNRTL\theatre_fle_11,theatre_fle_11.xml,NaN,theatre_fle_11.trs,NaN,NaN,theatre_fle_11
3,E:\Corpus\TCOF_CNRTL\telephone_lam_13,telephone_lam_13.xml,telephone_lam_13.wav,telephone_lam_13.trs,anonymise_telephone_lam_13.wav,anonymise_telephone_lam_13.trs,telephone_lam_13
4,E:\Corpus\TCOF_CNRTL\tel_maz_07,tel_maz_07.xml,tel_maz_07.wav,tel_maz_07.trs,NaN,NaN,tel_maz_07
...,...,...,...,...,...,...,...
584,E:\Corpus\TCOF_CNRTL\thibaut1_der,thibaut1_der.xml,thibaut1_der.wav,thibaut1_der.trs,NaN,NaN,thibaut1_der
585,E:\Corpus\TCOF_CNRTL\thibault1_lev,thibault1_lev.xml,thibault1_lev.wav,thibault1_lev.trs,NaN,NaN,thibault1_lev
586,E:\Corpus\TCOF_CNRTL\thibault1_cor,thibault1_cor.xml,thibault1_cor.wav,thibault1_cor.trs,NaN,NaN,thibault1_cor
587,E:\Corpus\TCOF_CNRTL\thibaud1_son,thibaud1_son.xml,thibaud1_son.wav,thibaud1_son.trs,NaN,NaN,thibaud1_son


# **Extraction des métadonnées (fichier .xml)**

1. Extrait toutes les métadonnées depuis le fichier .xml
2. Détecte si le fichier est encodé en utf8 ou iso, et le décode selon le format.
2. Output un dataframe avec pour chaque corpus, les métadonnées associées extraites depuis le fichier .xml

In [35]:
def detect_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding'] 
    # Open the file with the detected encoding
    with open(file, 'r', encoding=encoding) as f:
        soup = BeautifulSoup(f, "lxml-xml")
    return soup

In [49]:
def xml2df(xml_files):
    list_df =  []
    for xml_file in xml_files:
        soup = detect_encoding(xml_file)
        dict_file, locuteur = parsexml(soup)
        df_corpus = get_df(dict_file)
        locuteur_list = list(locuteur.items())
        df_corpus['locuteur'] = [locuteur_list] * len(df_corpus)
        list_df.append(df_corpus)
    df = pd.concat(list_df)
    return df

In [50]:
def get_df(my_dict):
    dict2df_corpus = my_dict.copy()
    df_corpus = pd.DataFrame([dict2df_corpus])
    return df_corpus

In [51]:
def get_general(soup, my_dict, general):
    for tag_name in general:
        tag = soup.find(tag_name)
        if tag is not None:
            tag_value = tag.text.strip()
            tag_name = tag.name
            my_dict[tag_name] = tag_value
    return my_dict

def get_enregistrement(soup, my_dict, enregistrement):
    for tag_name in enregistrement:
        tag = soup.find(tag_name)
        if tag is not None:
            tag_value = tag.text.strip()
            tag_name = tag.name
            my_dict[tag_name] = tag_value
    return my_dict

def get_referencement(soup, my_dict, referencement):
    for tag_name in referencement:
        tag = soup.find(tag_name)
        if tag is not None:
            tag_value = tag.text.strip()
            tag_name = tag.name
            my_dict[tag_name] = tag_value
    return my_dict

def get_transcription(soup, my_dict, transcription):
    for tag_name in transcription:
        tag = soup.find(tag_name)
        if tag is not None:
            tag_value = tag.text.strip()
            tag_name = tag.name
            my_dict[tag_name] = tag_value
    return my_dict

def get_locuteurs(soup, my_dict, locuteur_tag):
    # get all locuteurs
    locuteurs = soup.find_all('locuteur')

    # create a dictionary of locuteurs
    for locuteur in locuteurs[1:]:
        loc_principal = locuteur.get('locuteurPrincipal')
        loc_nb_tour = locuteur.get('nombre_tours')
        loc_identifiant = locuteur.get('identifiant')

        # create a new dictionary for this locuteur
        dict_locuteur = {}
        dict_locuteur['nb_tour'] = loc_nb_tour
        dict_locuteur['locuteur_principal'] = loc_principal

        for tag_name in locuteur_tag:
            tag = locuteur.find(tag_name)
            if tag is not None:
                tag_value = tag.text.strip()
                tag_name = tag.name
                dict_locuteur[tag_name] = tag_value
        # add this locuteur to the dictionary
        my_dict[loc_identifiant] = dict_locuteur
    return my_dict

In [52]:
def parsexml(soup):
    general = ["nomDossier", "responsable_corpus","droit_acces","lien_autre_corpus","mail",'logiciel_alignement','anonymisation',"nombre_locuteurs","relation","type_corpus","modalite_recueil","canal","cadre","degre",'situation_enonciation',"genre","support_dialogue","document_annexe","resume","commentaire","createur_fiche",'date_creation_fiche']
    enregistrement = ["nom_fichier", "responsable", "autorisation", "qualite","taille","duree","date","duree_transcription","debut_timecode_transcription", "dernier_timecode_transcription", "pays","region","ville","arrondissement", "description_lieu","format"]
    transcription = ["nom_fichier", "transcripteur","reviseur","format", "nombre_mots", "convention_transcription"]
    referencement = ["nom_corpus", "responsable", "titre", "laboratoire"]
    locuteur_tag = ['age','sexe','etude','formation','profession_actuelle','profession_anterieure','role','degre','statut_francais','autre_langue','relation_locuteur','naissance','residence','appartenance','particularite','nombre_mots','temps_parole']
    my_dict = {}
    my_dict.update(get_general(soup, my_dict, general))
    my_dict.update(get_enregistrement(soup, my_dict, enregistrement))
    my_dict.update(get_transcription(soup, my_dict,transcription))
    my_dict.update(get_referencement(soup, my_dict, referencement))
    locuteur = {}
    dict_locuteur = get_locuteurs(soup, locuteur, locuteur_tag)
    #my_dict.update(get_locuteurs(soup, my_dict, locuteur_tag))
    return my_dict, dict_locuteur

In [53]:
xml_files_cnrtl = [r'{}\{}.xml'.format(x, y).replace('\\','/') for x,y in list(zip(df_cnrtl['corpus'].tolist(), df_cnrtl['corpus_name'].tolist()))]

In [54]:
df_cnrtl_metadonnees = xml2df(xml_files_cnrtl)

In [55]:
#liste des métadonnées de chaque corpus
df_cnrtl_metadonnees

,nomDossier,responsable_corpus,droit_acces,mail,logiciel_alignement,anonymisation,nombre_locuteurs,relation,type_corpus,modalite_recueil,...,description_lieu,format,transcripteur,reviseur,nombre_mots,convention_transcription,nom_corpus,titre,laboratoire,locuteur
0,theorielinguistique_07,Virginie André,Libre,virginie.andre@univ-lorraine.fr,Transcriber,Son + transcription\nBip,3,professionnelle,entre adultes,Inconnue,...,,Wav,,,2423,2008,theorielinguistique_07,TCOF,ATILF,"[(L1, {'nb_tour': '66', 'locuteur_principal': ..."
0,testssncf_qab_12,Virginie André,Libre,virginie.andre@univ-lorraine.fr,Transcriber,Son + transcription\nBip,2,lien de parenté,entre adultes,Inconnue,...,,Wav,"MICHEL Annlyse, QABICE Fanny",NASSAU Guillaume,3086,2008,testssncf_qab_12,TCOF,ATILF,"[(L1, {'nb_tour': '70', 'locuteur_principal': ..."
0,theatre_fle_11,Virginie André,Libre,virginie.andre@univ-lorraine.fr,Transcriber,Son + transcription\nBip,8,lien amical,entre adultes,Inconnue,...,,Wav,"FLEURANCE Clémentine, THOMARDEL Camille, GIRAR...",NASSAU Guillaume,3859,2008,theatre_fle_11,TCOF,ATILF,"[(L1, {'nb_tour': '22', 'locuteur_principal': ..."
0,telephone_lam_13,Virginie André,Libre,virginie.andre@univ-lorraine.fr,Transcriber,Son + transcription\nBip,2,lien de parenté,entre adultes,Inconnue,...,,Wav,LAMBERT LUCIE,NASSAU Guillaume,2372,2008,telephone_lam_13,TCOF,ATILF,"[(L1, {'nb_tour': '90', 'locuteur_principal': ..."
0,tel_maz_07,Virginie André,Libre,Virginie.Andre@univ-nancy2.fr,Transcriber,Son + transcription\nBip,2,lien amical,entre adultes,Inconnue,...,,Wav,"Mazoyer Mathilde, Gabrion Julie",Stéphanie Houin,1169,2008,tel_maz_07,TCOF,ATILF,"[(L1, {'nb_tour': '46', 'locuteur_principal': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,thibaut1_der,Emmanuelle Canut,Libre,emmanuelle.canut@univ-lorraine.fr,Transcriber,Son + transcription\nBip,2,connaissance,adulte-enfant,Sollicitées par un chercheur,...,,Wav,Dereniowski Catherine,G. Nassau,2459,2008,thibaut1_der,TCOF,ATILF,"[(Adulte, {'nb_tour': '77', 'locuteur_principa..."
0,thibault1_lev,Emmanuelle Canut,Libre,emmanuelle.canut@univ-lorraine.fr,Transcriber,Son + transcription\nBip,2,connaissance,adulte-enfant,Sollicitées par un chercheur,...,,Wav,Y.Sow,E.Canut,940,2008,thibault1_lev,TCOF,ATILF,"[(Thibault, {'nb_tour': '71', 'locuteur_princi..."
0,thibault1_cor,Emmanuelle Canut,Libre,emmanuelle.canut@univ-lorraine.fr,Transcriber,Son + transcription\nBip,2,connaissance,adulte-enfant,Sollicitées par un chercheur,...,,Wav,Y.Sow,E.Canut,728,2008,thibault1_cor,TCOF,ATILF,"[(Adulte, {'nb_tour': '88', 'locuteur_principa..."
0,thibaud1_son,Emmanuelle Canut,Libre,emmanuelle.canut@univ-lorraine.fr,Transcriber,Son + transcription\nBip,2,connaissance,adulte-enfant,Sollicitées par un chercheur,...,,Wav,"Huguin, Sontag Eugénie",G. Nassau,1067,2008,thibaud1_son,TCOF,ATILF,"[(Adulte, {'nb_tour': '55', 'locuteur_principa..."


# **Analyse des tag.names pour les fichier .trs**

1. Récupère tous les noms des balises utilisées dans le fichier trs
2. Compare les noms utilisés dans tous les corpus
3. Output un dataframe avec pour chaque corpus, le fichier .trs correspondant, tous les tags utilisés, la différence avec les autres corpus

In [18]:
def get_all_tagsname(xml_file):
    # Parse the XML file into an ElementTree object
    try : 
        tree = ET.parse(xml_file)
    # Get the root element of the tree
        root = tree.getroot()
    # Initialize an empty set to store the tag names
        tag_names = set()
    # Iterate over all elements in the tree and add their tag names to the set
        for elem in root.iter():
            tag_names.add(elem.tag)
    # Return the set of tag names
    except: 
        return "Pas de fichier trouvé à cet emplacement"
    return tag_names

In [19]:
def get_corpus_name(path):
    filename = re.search(r'[^\\\/]*?(?=\.\w+$)', path).group()
    return filename

In [20]:
def get_df_tags(xml_files):
    corpus_name = [get_corpus_name(xml_file) for xml_file in xml_files]
    tags_ = [get_all_tagsname(xml_file) for xml_file in xml_files]
    df = pd.DataFrame(list(zip(corpus_name, tags_)), columns = ['corpus_name','tags'])
    return df

In [21]:
def get_difference(df):
    # prendre la liste des tags + l'intersection de toutes les listes
    lists_tags = df['tags'].tolist()
    results_union = set().union(*lists_tags)
    # trouver les différences entre l'intersection de toutes les listes et la liste des tags d'un corpus donné
    df['is_same'] = df['tags'].apply(lambda x:is_same(results_union,x))
    return results_union, df

In [22]:
def is_same(list1, list2):
    s = set(list1)
    difference = [x for x in list2 if x not in s]
    if difference : 
        return difference
    else: 
        return True

In [23]:
trs_files_cnrtl = [r'{}\{}.trs'.format(x, y).replace('\\','/') for x,y in list(zip(df_cnrtl['corpus'].tolist(), df_cnrtl['corpus_name'].tolist()))]

In [24]:
df_cnrtl_tagstrs = get_df_tags(trs_files_cnrtl)

Comparaison des tags

In [26]:
results_union_trs, df_cnrtl_tagstrs = get_difference(df_cnrtl_tagstrs)
print('Nb de tags communs :', len(results_union_trs), '\n Liste des tags communs :\n', results_union_trs)

Nb de tags communs : 12 
 Liste des tags communs :
 {'Sync', 'Trans', 'Topic', 'Turn', 'Section', 'Episode', 'Topics', 'Speaker', 'Who', 'Comment', 'Event', 'Speakers'}


In [27]:
df_cnrtl_tagstrs

,corpus_name,tags,is_same
0,theorielinguistique_07,"{Sync, Trans, Speaker, Turn, Section, Comment,...",True
1,testssncf_qab_12,"{Sync, Trans, Speaker, Turn, Section, Comment,...",True
2,theatre_fle_11,"{Sync, Trans, Speaker, Turn, Section, Comment,...",True
3,telephone_lam_13,"{Sync, Trans, Speaker, Turn, Section, Episode,...",True
4,tel_maz_07,"{Sync, Trans, Speaker, Turn, Section, Comment,...",True
...,...,...,...
584,thibaut1_der,"{Sync, Topics, Trans, Speaker, Topic, Turn, Se...",True
585,thibault1_lev,"{Sync, Topics, Trans, Speaker, Topic, Turn, Se...",True
586,thibault1_cor,"{Sync, Topics, Trans, Speaker, Topic, Turn, Se...",True
587,thibaud1_son,"{Sync, Topics, Trans, Speaker, Topic, Turn, Se...",True


Tous les fichiers trs ont les mêmes tags

# Parsage fichier .trs 

1. extrait toutes les données du fichier .trs (possible de récupérer également des données qui n'apparaissent pas dans le dataframe final, comme le topic)
2. Output un dataframe pour chaque fichier .trs avec à chaque ligne un tour de parole, son début, sa fin, le locuteur et ses caractéristiques, le fichier audio, l'encoding et *le texte synchronisé*

*NB : le texte synchronisé n'est pas encore optimal, j'ai du mal à récupérer le texte entre deux balises Sync, car d'autres balises Event sont aussi utilisées + l'encodage est tjrs en iso*

In [118]:
def detect_encoding2(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding'] 
    # Open the file with the detected encoding
    with open(file, 'r', encoding=encoding) as f:
        soup = BeautifulSoup(f, "lxml-xml")
    return soup, encoding

In [346]:
def parse_trs2(file):
    transcription = ["nom_fichier", "transcripteur","reviseur","format", "nombre_mots", "convention_transcription"]
    referencement = ["nom_corpus", "responsable", "titre", "laboratoire"]
    locuteur_tag = ['age','sexe','etude','formation','profession_actuelle','profession_anterieure','role','degre','statut_francais','autre_langue','relation_locuteur','naissance','residence','appartenance','particularite','nombre_mots','temps_parole']
    soup, encoding = detect_encoding2(file)
    my_dict = {}
    dict_trans, audio_filename = get_trans2(soup, my_dict)
    my_dict.update(dict_trans)  
    dict_loc = get_speaker2(soup, my_dict)
    my_dict.update(dict_loc)
    my_dict.update(get_topics2(soup, my_dict))
    my_dict.update(get_section2(soup, my_dict))
    dict_turns = get_turn2(soup)
    df = pd.DataFrame.from_dict(dict_turns,orient='index')
    df['audio_filename']= audio_filename
    df['encoding']=encoding
    return df

In [347]:
def get_trans2(soup, my_dict):
    trans = soup.find('Trans')
    # create a dictionary of locuteurs
    if trans is not None:
        audio_filename = trans.get('audio_filename')
        my_dict['audio_filename'] = audio_filename
    return my_dict, audio_filename

def get_speaker2(soup, my_dict):
    speakers = soup.find_all('Speaker')
    # create a dictionary of speakers
    for speaker in speakers:
        scope = speaker.get('scope')
        accent = speaker.get('accent')
        dialect = speaker.get('dialect')
        check = speaker.get('check')
        name = speaker.get('name')
        id_ = speaker.get('id')
        # create a new dictionary for this speaker
        dict_locuteur = {}
        dict_locuteur['scope'] = scope
        dict_locuteur['accent'] = accent
        dict_locuteur['dialect'] = dialect
        dict_locuteur['check'] = check
        dict_locuteur['name'] = name
        dict_locuteur['id'] = id_
        my_dict[id_] = dict_locuteur
    return my_dict

def get_topics2(soup, my_dict):
    topics = soup.find_all('Topic')
    # create a dictionary of locuteurs
    for topic in topics:
        id_ = topic.get('id')
        desc = topic.get('desc')
        # create a new dictionary for this speaker
        dict_topic = {}
        dict_topic['desc'] = desc
        dict_topic['id'] = id_
        my_dict[id_] = dict_topic
    return my_dict

def get_section2(soup, my_dict):
    sections = soup.find_all('Section')
    # create a dictionary of locuteurs
    for section in sections:
        topic = section.get('topic')
        endTime = section.get('endTime')
        startTime = section.get('startTime')
        type_ = section.get('type')
        # create a new dictionary for this speaker
        dict_section = {}
        dict_section['topic'] = topic
        dict_section['endTime'] = endTime
        dict_section['startTime'] = startTime
        dict_section['type'] = type_
        my_dict['section_{}'.format(topic)] = dict_section
    return my_dict

def get_turn2(soup):
    turns = soup.find_all('Turn')
    dict_turns = {}
    dict_loc = {}
    dict_loc.update(get_speaker2(soup, dict_loc))
    for index, turn in enumerate(turns):
        endTime = turn.get('endTime')
        startTime = turn.get('startTime')
        speaker = turn.get('speaker')
        sync_tags = turn.find_all('Sync')
        result = get_sync_times(turn)
        text = turn.text.encode('iso-8859-1').decode('utf-8')
        dict_turn = {}
        dict_turn['startTime']=startTime
        dict_turn['endTime']=endTime
        dict_turn['speaker'] = speaker
        dict_turn['speaker_characteristic'] = dict_loc.get('{}'.format(speaker))
        dict_turn['text']= text.replace('\n','')
        dict_turn['text_synchronisé'] = result
        dict_turns['{}'.format(index)] = dict_turn
    return dict_turns

In [348]:
def get_sync_times(turn):
    sync_tags = turn.find_all('Sync')
    sync_times = [float(sync_tag['time']) for sync_tag in sync_tags]
    text = turn.get_text().replace('\n\n','\n').strip().splitlines()
    result = [(sync_time, text) for sync_time, text in list(zip(sync_times, text))]
    return result

In [362]:
list_df_turn = [parse_trs2(file) for file in trs_files_cnrtl]

In [363]:
list_df_turn[8]

,startTime,endTime,speaker,speaker_characteristic,text,text_synchronisé,audio_filename,encoding
0,0,3.73,spk1,"{'scope': 'local', 'accent': '', 'dialect': 'n...",et au niveau des tenues pour le sport il y en ...,"[(0.0, et au niveau des tenues pour le sport i...",sports_pet_07.wav,ISO-8859-1
1,3.73,47.069,spk2,"{'scope': 'local', 'accent': '', 'dialect': 'n...",euh alors euh déjà j'ai jamais compris pourquo...,"[(3.73, euh alors euh dÃ©jÃ j'ai jamais compr...",sports_pet_07.wav,ISO-8859-1
2,47.069,54.832,spk1,"{'scope': 'local', 'accent': '', 'dialect': 'n...",mais euh comment je veux te dire à porter enfi...,"[(47.069, mais euh comment je veux te dire Ã ...",sports_pet_07.wav,ISO-8859-1
3,54.832,56.053,spk2,"{'scope': 'local', 'accent': '', 'dialect': 'n...",euh,"[(54.832, euh)]",sports_pet_07.wav,ISO-8859-1
4,56.053,58.788,spk1,"{'scope': 'local', 'accent': '', 'dialect': 'n...",porter un kimono ou euh quand tu fais du sport,"[(56.053, porter un kimono ou euh quand tu fai...",sports_pet_07.wav,ISO-8859-1
...,...,...,...,...,...,...,...,...
120,594.411,596.56,spk1,"{'scope': 'local', 'accent': '', 'dialect': 'n...",mais euh les < les Italiens avec,"[(594.411, mais euh les < les Italiens avec)]",sports_pet_07.wav,ISO-8859-1
121,596.56,603.169,spk2,"{'scope': 'local', 'accent': '', 'dialect': 'n...",les plongeons > toutes les deux secondes à pei...,"[(596.56, les plongeons > toutes les deux seco...",sports_pet_07.wav,ISO-8859-1
122,603.169,605.272,spk1,"{'scope': 'local', 'accent': '', 'dialect': 'n...",même le Brésilien Ronaldo hein,"[(603.169, mÃªme le BrÃ©silien Ronaldo hein)]",sports_pet_07.wav,ISO-8859-1
123,605.272,605.873,spk2,"{'scope': 'local', 'accent': '', 'dialect': 'n...",oui,"[(605.272, oui)]",sports_pet_07.wav,ISO-8859-1
